## Chinook database
![Chinook Schema](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)

### Connect jupyter notebook to the databse

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

### Explore table and views

In [2]:
%%sql
SELECT
    name, 
    Type
FROM sqlite_master
WHERE type IN ("table", "view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### Number of records sold for each genre in USA

In [3]:
%%sql
With track_invoice AS
(
    SELECT g.name, il.quantity 
        FROM track t
        INNER JOIN invoice_line il
        ON t.track_id = il.track_id
        INNER JOIN invoice i
        ON i.invoice_id = il.invoice_id
        INNER JOIN genre g
        ON g.genre_id = t.genre_id
        WHERE i.billing_country = "USA"
)
SELECT 
    name, 
    sum(quantity) as num_of_tracks
FROM track_invoice
GROUP BY name
ORDER BY num_of_tracks DESC;

Done.


name,num_of_tracks
Rock,561
Alternative & Punk,130
Metal,124
R&B/Soul,53
Blues,36
Alternative,35
Latin,22
Pop,22
Hip Hop/Rap,20
Jazz,14


Rock, Alternative & Punk, Metal, R&B/Soul and Blues are the top 5 mostly sold genre in the USA.
Artist Name	Genre
Regal	Hip-Hop
Red Tone	Punk
Meteor and the Girls	Pop
Slim Jim Bites	Blues
So for the above new albums, Slim Jim Bites is expected to perfrom much better in USA than other artists

Total dollar amount of sales assigned to each sales support agent within the company

###  Total dollar amount of sales assigned to each sales support agent within the company

In [4]:
%%sql
With employee_sales AS
(
    SELECT e.first_name, e.last_name, i.total
        FROM employee e
            LEFT JOIN
                customer c ON e.employee_id = support_rep_id
                    LEFT JOIN
                        invoice i ON i.customer_id = c.customer_id
)
SELECT 
    first_name || " " || last_name AS name,
    sum(total) as sales_made
FROM employee_sales
GROUP BY name
ORDER BY sales_made DESC;
    

Done.


name,sales_made
Jane Peacock,1731.510000000004
Margaret Park,1584.0000000000032
Steve Johnson,1393.9200000000028
Andrew Adams,None
Laura Callahan,None
Michael Mitchell,None
Nancy Edwards,None
Robert King,None


Jane Peacock, Margaret Park and Steve Johnson are only employees who were able to make sales while others were unable to make sales. Jane is the most hard workign eployee.

### Information on purchase for different countries

In [ ]:
%%sql

WITH customer_invoice 
    AS (
        SELECT * 
        FROM customer c
        LEFT JOIN 
            (
                SELECT customer_id, sum(total) as total,
                avg(total) as avg_order
                FROM invoice
                GROUP BY customer_id
            )
        i ON c.customer_id = i.customer_id
    )
SELECT 
    new_country,
    sum(num) as total_customer,
    sum(total) as total_sales,
    total/num as avg_sales_per_customer,
    avg(avg_order) as avg_order
    FROM
    (
    SELECT
        CASE WHEN num = 1 THEN 'Other'
            ELSE country end AS new_country,
            num, total, avg_order
        FROM
            (
                SELECT country, 
                    count(*) as num,
                    sum(total) as total,
                    avg(avg_order) as avg_order
                    FROM customer_invoice
                    GROUP BY country
            )
    )
    GROUP BY new_country
    ORDER BY CASE WHEN num = 1 then 0
    ELSE 1 END DESC, total_sales DESC;



There are more customers in USA but this also shows the fact that the per person spending is less in India

### Total number of invoices in which all the tracks present in the album were purchased or not

In [45]:
%%sql
With invoice_track AS
(
SELECT a.invoice_id, b.album_id, COUNT(DISTINCT a.track_id) AS invoice_track_num
    FROM invoice_line a
    INNER JOIN 
    track b
    USING(track_id)
    GROUP BY a.invoice_id, b.album_id
),
album_track AS
(
    SELECT album_id, COUNT(track_id) as album_track_num
    FROM album
    LEFT JOIN
    track 
    USING(album_id)
    GROUP BY album_id
),
purchase AS
(
    SELECT CASE WHEN album_track_num = invoice_track_num THEN "Full album"
    ELSE "Individual track" END AS purchase_type,
    invoice_id
    FROM album_track 
    LEFT JOIN
    invoice_track
    USING(album_id)
)
SELECT purchase_type , COUNT(DISTINCT invoice_id) AS invoice_num,
CAST(COUNT(DISTINCT invoice_id) AS float)*100/(SELECT COUNT( DISTINCT invoice_id) FROM purchase) AS pert_of_invoice
FROM purchase
GROUP BY purchase_type;

Done.


purchase_type,invoice_num,pert_of_invoice
Full album,201,32.73615635179153
Individual track,500,81.43322475570032


Most of the purchase made were individual track purchase. So it is better to change our strategry of focusing on selling the top popular tracks from the given album than the whole list of tracks for that albu,

### Which artist is used in the most playlists?

In [65]:
%%sql
WITH playlist_info AS
(
    SELECT a.playlist_id, a.name, b.track_id
    FROM playlist a 
    LEFT JOIN 
    playlist_track b
    USING(playlist_id)
),
artist_info AS
(
    SELECT a.artist_id, a.name, b.album_id
    FROM artist a
    LEFT JOIN
    album b 
    USING(artist_id)
),
all_info AS
(
    SELECT a.name AS playlist, b.name AS artist, c.track_id
    FROM playlist_info a
    LEFT JOIN 
    track c
    ON a.track_id = c.track_id
    LEFT JOIN artist_info b
    ON
    b.album_id = c.album_id
    
)
SELECT artist, COUNT(distinct playlist) AS num_times_artist_present,
COUNT(playlist) as num_of_features
    FROM all_info
    GROUP BY artist
    ORDER BY num_times_artist_present DESC;



Done.


artist,num_times_artist_present,num_of_features
Eugene Ormandy,6,15
Academy of St. Martin in the Fields & Sir Neville Marriner,5,10
Berliner Philharmoniker & Herbert Von Karajan,5,13
English Concert & Trevor Pinnock,5,9
The King's Singers,5,10
"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",4,5
Adrian Leaper & Doreen de Feis,4,5
Alberto Turco & Nova Schola Gregoriana,4,5
Antal Doráti & London Symphony Orchestra,4,5
Barry Wordsworth & BBC Concert Orchestra,4,5


Eugene Ormandy is present inthe most playlist but this doesn't account for the popularity. Iron maiden though present in less number of playlist has the highest number of songs present. This might be due to the generation gap where younger people have more songs of iron maiden.

### How many tracks have been purchased vs not purchased?

In [70]:
%%sql 
WITH track_info AS
(SELECT track.track_id as album_track, 
    i.track_id  as invoice_track
    FROM track
    LEFT JOIN
(    
    SELECT DISTINCT track_id
    FROM invoice
    LEFT JOIN
    invoice_line
    USING(invoice_id)
) i
USING(track_id)),
purchase AS
(
    SELECT
    CASE WHEN album_track = invoice_track then "Purchased"
    ELSE "Not purchased" END as track_status
    FROM track_info
)
SELECT track_status, 
        COUNT(track_status) AS num_track
FROM purchase
GROUP BY track_status;



Done.


track_status,num_track
Not purchased,1697
Purchased,1806


### Is the range of tracks in the store reflective of their sales popularity?

In [76]:
%%sql
SELECT genre.name AS track_range, count(genre.name) AS num_tracks
    FROM track
    LEFT JOIN
    genre
    USING(genre_id)
    GROUP BY genre.name
    ORDER BY num_tracks DESC;

Done.


track_range,num_tracks
Rock,1297
Latin,579
Metal,374
Alternative & Punk,332
Jazz,130
TV Shows,93
Blues,81
Classical,74
Drama,64
R&B/Soul,61


Rock, Metal and Latin are the 3 genres for which we have more tracks in the store

In [86]:
%%sql
SELECT genre.name as genre_type,
        SUM(invoice.total) AS total_sales,
        SUM(invoice.total)/COUNT(invoice_line.track_id) AS sales_per_track,
        COUNT(DISTINCT invoice_line.track_id) AS track_sold,
        COUNT(track.track_id) AS total_tracks



    FROM track 
    LEFT JOIN invoice_line
    USING(track_id)
    LEFT JOIN invoice
    USING(invoice_id)
    LEFT JOIN genre
    USING(genre_id)
    GROUP BY genre.name
    ORDER BY sales_per_track DESC;

Done.


genre_type,total_sales,sales_per_track,track_sold,total_tracks
Hip Hop/Rap,463.32000000000005,14.040000000000001,21,47
Easy Listening,951.3899999999994,12.856621621621613,24,74
Electronica/Dance,614.7899999999998,11.177999999999997,29,56
Blues,1379.0700000000038,11.121532258064546,56,149
R&B/Soul,1751.310000000005,11.014528301886823,55,165
Jazz,1302.8400000000004,10.767272727272731,61,190
Latin,1705.770000000001,10.214191616766472,119,627
Rock,26751.77999999988,10.152478178368076,915,3017
TV Shows,19.8,9.9,2,93
Alternative & Punk,4841.099999999979,9.839634146341421,176,648


Most sales are coming from the HIP/HOP. So we should have more of the popular genres in the store. Also some of the genres have been unsold at all. So such genres track should not be restocked.